# W266 Project

### Adam Sayre & Erin Werner

## CNN Model

In [1]:
import sys
sys.path.insert(0,r"./anaconda3/lib/python3.7/site-packages")

In [2]:
import numpy as np
import csv
import pandas as pd 
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import importlib
import emoji
import tensorflow as tf
import nltk
import re
from nltk.corpus import brown
nltk.download('stopwords')
from nltk.corpus import stopwords
assert(nltk.download("treebank"))
from nltk.corpus import europarl_raw
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package treebank to /home/ubuntu/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.backend import clear_session

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [5]:
from pickle import load
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

In [6]:
data = pd.read_csv("tweet_data.csv") 
data.head()

,Emotion,Content,Original Content
0,disappointed,oh fuck did i wrote fil grinningfacewithsweat ...,b'RT @Davbingodav: @mcrackins Oh fuck.... did ...
1,disappointed,i feel nor am i shamed by it,i feel nor am i shamed by it
2,disappointed,i had been feeling a little bit defeated by th...,i had been feeling a little bit defeated by th...
3,happy,imagine if that reaction guy that called jj kf...,"b""@KSIOlajidebt imagine if that reaction guy t..."
4,disappointed,i wouldnt feel burdened so that i would live m...,i wouldnt feel burdened so that i would live m...


In [7]:
data_e = pd.read_csv("dataset(clean)_e.csv") 
data_e.head()[['Emotion','Content','Original Content','E_Content']]

,Emotion,Content,Original Content,E_Content
0,disappointed,oh fuck did i wrote fil grinningfacewithsweat ...,b'RT @Davbingodav: @mcrackins Oh fuck.... did ...,rt usertaginstance usertaginstance oh fuck wro...
1,disappointed,i feel nor am i shamed by it,i feel nor am i shamed by it,feel shamed
2,disappointed,i had been feeling a little bit defeated by th...,i had been feeling a little bit defeated by th...,feeling little bit defeated steps faith would ...
3,happy,imagine if that reaction guy that called jj kf...,"b""@KSIOlajidebt imagine if that reaction guy t...",usertaginstance imagine reaction guy called jj...
4,disappointed,i wouldnt feel burdened so that i would live m...,i wouldnt feel burdened so that i would live m...,wouldnt feel burdened would live life testamen...


In [8]:
data_a = pd.read_csv("dataset(clean)_a.csv") 
data_a.head()[['Emotion','Content','Original Content','A_Content']]

,Emotion,Content,Original Content,A_Content
0,disappointed,oh fuck did i wrote fil grinningfacewithsweat ...,b'RT @Davbingodav: @mcrackins Oh fuck.... did ...,brt davbingodav mcrackins oh fuck did i wrote ...
1,disappointed,i feel nor am i shamed by it,i feel nor am i shamed by it,i feel nor am i shamed by it
2,disappointed,i had been feeling a little bit defeated by th...,i had been feeling a little bit defeated by th...,i had been feeling a little bit defeated by th...
3,happy,imagine if that reaction guy that called jj kf...,"b""@KSIOlajidebt imagine if that reaction guy t...",bksiolajidebt imagine if that reaction guy tha...
4,disappointed,i wouldnt feel burdened so that i would live m...,i wouldnt feel burdened so that i would live m...,i wouldnt feel burdened so that i would live m...


In [9]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

### Shallow CNN

#### Original Cleaned Data

In [10]:
possible_labels = data.Emotion.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'disappointed': 0, 'happy': 1, 'angry': 2}

In [11]:
data['label'] = data.Emotion.replace(label_dict)

In [12]:
# X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(data.Content.values, data.label.values, test_size=0.3, 
#                                                   random_state=42, stratify=data.label.values)

#X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_train_c, y_train_c, test_size=0.5, 
#                                                  random_state=42, stratify=y_train_c)

In [13]:
# vectorizer_c = CountVectorizer()
# vectorizer_c.fit(X_train_c)

# X_train_c = vectorizer_c.transform(X_train_c)
# X_val_c  = vectorizer_c.transform(X_val_c)

#### Original Uncleaned Data

In [14]:
data["Original_Content"] = data["Original Content"]

In [15]:
# X_train_oc, X_val_oc, y_train_oc, y_val_oc = train_test_split(data.Original_Content.values, data.label.values, 
#                                                   test_size=0.3, random_state=42, stratify=data.label.values)

#X_train_oc, X_val_oc, y_train_oc, y_val_oc = train_test_split(X_train_oc, y_train_oc, test_size=0.5, 
#                                                  random_state=42, stratify=y_train_oc)

In [16]:
# vectorizer_oc = CountVectorizer()
# vectorizer_oc.fit(X_train_oc)

# X_train_oc = vectorizer_oc.transform(X_train_oc)
# X_val_oc  = vectorizer_oc.transform(X_val_oc)
# X_train_oc

#### Custom Cleaned Data #1

In [17]:
possible_labels_e = data_e.Emotion.unique()

label_dict_e = {}
for index, possible_label in enumerate(possible_labels_e):
    label_dict_e[possible_label] = index

data_e['label'] = data_e.Emotion.replace(label_dict_e)

In [18]:
# X_train_e, X_val_e, y_train_e, y_val_e = train_test_split(data_e.E_Content.values, data_e.label.values, 
#                                                   test_size=0.3,random_state=42, stratify=data_e.label.values)

#X_train_e, X_val_e, y_train_e, y_val_e = train_test_split(X_train_e, y_train_e, test_size=0.5, 
#                                                  random_state=42, stratify=y_train_e)

In [19]:
# vectorizer_e = CountVectorizer()
# vectorizer_e.fit(X_train_e)

# X_train_e = vectorizer_e.transform(X_train_e)
# X_val_e  = vectorizer_e.transform(X_val_e)
# X_train_e

#### Custom Cleaned Data #2

In [20]:
possible_labels_a = data_a.Emotion.unique()

label_dict_a = {}
for index, possible_label in enumerate(possible_labels_a):
    label_dict_a[possible_label] = index

data_a['label'] = data_a.Emotion.replace(label_dict_a)

In [21]:
# X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(data_a.A_Content.values, data_a.label.values, 
#                                                   test_size=0.3,random_state=42, stratify=data_a.label.values)

#X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X_train_a, y_train_a, test_size=0.5, 
#                                                  random_state=42, stratify=y_train_a)

In [22]:
# vectorizer_a = CountVectorizer()
# vectorizer_a.fit(X_train_a)

# X_train_a = vectorizer_a.transform(X_train_a)
# X_val_a  = vectorizer_a.transform(X_val_a)
# X_train_a

### Deep CNN 

In [23]:
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])

# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [24]:
# define the model
def define_model(length, vocab_size):
    # channel 1
    inputs1 = layers.Input(shape=(length,))
    embedding1 = layers.Embedding(vocab_size, 100)(inputs1)
    conv1 = layers.Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = layers.Dropout(0.5)(conv1)
    pool1 = layers.MaxPooling1D(pool_size=2)(drop1)
    flat1 = layers.Flatten()(pool1)
    # channel 2
    inputs2 = layers.Input(shape=(length,))
    embedding2 = layers.Embedding(vocab_size, 100)(inputs2)
    conv2 = layers.Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = layers.Dropout(0.5)(conv2)
    pool2 = layers.MaxPooling1D(pool_size=2)(drop2)
    flat2 = layers.Flatten()(pool2)
    # channel 3
    inputs3 = layers.Input(shape=(length,))
    embedding3 = layers.Embedding(vocab_size, 100)(inputs3)
    conv3 = layers.Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = layers.Dropout(0.5)(conv3)
    pool3 = layers.MaxPooling1D(pool_size=2)(drop3)
    flat3 = layers.Flatten()(pool3)
    # merge
    merged = layers.concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = layers.Dense(10, activation='relu')(merged)
    outputs = layers.Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

#### Original Cleaned Data

In [38]:
X_train_tc, X_val_tc, y_train_tc, y_val_tc = train_test_split(data.Content.values, data.label.values, test_size=0.3, 
                                                  random_state=42, stratify=data.label.values)

#X_train_tc, X_val_tc, y_train_tc, y_val_tc = train_test_split(X_train_tc, y_train_tc, test_size=0.5, 
#                                                  random_state=42, stratify=y_train_tc)

In [39]:
# create tokenizer
tokenizer = create_tokenizer(X_train_tc)
# calculate max document length
length = max_length(X_train_tc)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# encode data
trainX_tc = encode_text(tokenizer, X_train_tc, length)
valX_tc = encode_text(tokenizer, X_val_tc, length)
print(trainX_tc.shape, valX_tc.shape)

Max document length: 119
Vocabulary size: 137547
(641602, 119) (274973, 119)


In [40]:
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX_tc,trainX_tc,trainX_tc], y_train_tc, epochs=10, batch_size=10)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 119)]        0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 119)]        0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 119)]        0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 119, 100)     13754700    input_10[0][0]                   
____________________________________________________________________________________________

KeyboardInterrupt: 

In [ ]:
dcnn_loss_tc, dcnn_accuracy_tc = model.evaluate([valX_tc,valX_tc,valX_tc], y_val_tc, verbose=False)

#### Original Uncleaned Data

In [45]:
# X_train_toc, X_val_toc, y_train_toc, y_val_toc = train_test_split(data.Original_Content.values, 
#                                                   data.label.values, test_size=0.3, 
#                                                   random_state=42, stratify=data.label.values)

X_train_toc, X_val_toc, y_train_toc, y_val_toc = train_test_split(X_train_toc, y_train_toc, test_size=0.75, 
                                                 random_state=42, stratify=y_train_toc)

In [46]:
# create tokenizer
tokenizer = create_tokenizer(X_train_toc)
# calculate max document length
length = max_length(X_train_toc)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# encode data
trainX_toc = encode_text(tokenizer, X_train_toc, length)
valX_toc = encode_text(tokenizer, X_val_toc, length)
print(trainX_toc.shape, valX_toc.shape)

Max document length: 80
Vocabulary size: 172961
(160400, 80) (481202, 80)


In [48]:
# define model
model = define_model(length, vocab_size)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit([trainX_toc,trainX_toc,trainX_toc], y_train_toc, epochs=10, batch_size=10)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 80)]         0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 80)]         0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 80, 100)      17296100    input_16[0][0]                   
____________________________________________________________________________________________

In [ ]:
dcnn_loss_toc, dcnn_accuracy_toc = model.evaluate([valX_toc,valX_toc,valX_toc], y_val_toc, verbose=False)

#### Custom Cleaned Data #1

In [29]:
X_train_te, X_val_te, y_train_te, y_val_te = train_test_split(data_e.E_Content.values, 
                                                  data_e.label.values, test_size=0.3, 
                                                  random_state=42, stratify=data_e.label.values)

#X_train_te, X_val_te, y_train_te, y_val_te = train_test_split(X_train_te, y_train_te, test_size=0.5, 
#                                                  random_state=42, stratify=y_train_te)

In [30]:
# create tokenizer
tokenizer = create_tokenizer(X_train_te)
# calculate max document length
length = max_length(X_train_te)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# encode data
trainX_te = encode_text(tokenizer, X_train_te, length)
valX_te = encode_text(tokenizer, X_val_te, length)
print(trainX_te.shape, valX_te.shape)

Max document length: 338
Vocabulary size: 248549
(641602, 338) (274973, 338)


In [31]:
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX_te,trainX_te,trainX_te], y_train_te, epochs=10, batch_size=10)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 338)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 338)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 338)]        0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 338, 100)     24854900    input_4[0][0]                    
____________________________________________________________________________________________

In [32]:
dcnn_loss_te, dcnn_accuracy_te = model.evaluate([valX_te,valX_te,valX_te], y_val_te, verbose=False)

#### Custom Cleaned Data #2

In [33]:
X_train_ta, X_val_ta, y_train_ta, y_val_ta = train_test_split(data_a.A_Content.values, 
                                                  data_a.label.values, test_size=0.3, 
                                                  random_state=42, stratify=data_a.label.values)

#X_train_ta, X_val_ta, y_train_ta, y_val_ta = train_test_split(X_train_ta, y_train_ta, test_size=0.5, 
#                                                  random_state=42, stratify=y_train_ta)

In [34]:
# create tokenizer
tokenizer = create_tokenizer(X_train_ta)
# calculate max document length
length = max_length(X_train_ta)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# encode data
trainX_ta = encode_text(tokenizer, X_train_ta, length)
valX_ta = encode_text(tokenizer, X_val_ta, length)
print(trainX_ta.shape, valX_ta.shape)

Max document length: 110
Vocabulary size: 606626
(641602, 110) (274973, 110)


In [35]:
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX_ta,trainX_ta,trainX_ta], y_train_ta, epochs=10, batch_size=10)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 110)]        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 110)]        0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 110)]        0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 110, 100)     60662600    input_7[0][0]                    
____________________________________________________________________________________________

In [36]:
dcnn_loss_ta, dcnn_accuracy_ta = model.evaluate([valX_ta,valX_ta,valX_ta], y_val_ta, verbose=False)

#### Results

In [37]:
dcnn_acc_train = [dcnn_accuracy_toc, dcnn_accuracy_tc, dcnn_accuracy_te, dcnn_accuracy_ta]
dcnn_loss_train = [dcnn_loss_toc, dcnn_loss_tc, dcnn_loss_te, dcnn_loss_ta]
dcnn_values = ['Orig. Uncleaned', 'Orig. Cleaned', 'Custom Cleaned #1', 'Custom Cleaned #2']
dcnn_df = pd.DataFrame()
dcnn_df['Cleaning Method'] = dcnn_values
dcnn_df['F1 Score'] = dcnn_acc_train
dcnn_df['Loss'] = dcnn_loss_train
dcnn_df

NameError: name 'dcnn_accuracy_tc' is not defined

In [42]:
# Partial Results just for deep cnn
dcnn_acc_train = [dcnn_accuracy_toc, dcnn_accuracy_te, dcnn_accuracy_ta]
dcnn_loss_train = [dcnn_loss_toc, dcnn_loss_te, dcnn_loss_ta]
dcnn_values = ['Orig. Uncleaned', 'Custom Cleaned #1', 'Custom Cleaned #2']
dcnn_df = pd.DataFrame()
dcnn_df['Cleaning Method'] = dcnn_values
dcnn_df['F1 Score'] = dcnn_acc_train
dcnn_df['Loss'] = dcnn_loss_train
dcnn_df

,Cleaning Method,F1 Score,Loss
0,Orig. Uncleaned,0.345274,-8.945458e+14
1,Custom Cleaned #1,0.336371,-6.082689e+14
2,Custom Cleaned #2,0.334160,-1.019285e+14
